In [1]:
!pip install pandas seaborn matplotlib scikit-learn gensim eli5 tiktoken spacy gensim

In [2]:
import tiktoken
import pandas as pd
from nltk.tokenize import word_tokenize, sent_tokenize
import numpy
import matplotlib.pyplot as plt
import re

In [3]:
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/florinekieraga/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/florinekieraga/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
df = pd.read_csv('training.1600000.processed.noemoticon.csv',
                 encoding = 'latin',header=None)


In [6]:
df.rename(columns={0: 'polarity', 1: 'id', 2: 'date', 3: 'query', 4: 'user', 5: 'text'}, inplace= True)
df.head()

,polarity,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


## Training Word2Vec and tf-idf 

In [7]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors


ImportError: cannot import name 'triu' from 'scipy.linalg' (/home/florinekieraga/.virtualenvs/nlp-projet/lib/python3.10/site-packages/scipy/linalg/__init__.py)

In [ ]:
X_list = list(df.)
model = 